# Data load

In [3]:
# pip install yfinance pandas requests beautifulsoup4

# 종목 list

In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class SimpleCryptoDownloader:
    def __init__(self, start_date="2015-01-01", end_date="2025-12-30", output='/workspace/AI모델/projects/coin/data/1h', top_n=20):
        self.top_n = top_n
        self.start_date = start_date
        self.end_date = end_date
        self.output = output
        self.symbols = []
        self.crypto_data = {}
        self.crypto_info = {}
        self.failed_symbols = []
        
    def get_top_crypto_symbols(self):
        """야후 파이낸스에서 시가총액 기준 상위 코인 티커 가져오기"""
        print(f"🔍 시가총액 기준 상위 {self.top_n}개 코인 티커 검색 중...")
        
        # 주요 코인들의 야후 파이낸스 티커 (USD 기준)
        major_cryptos = [
            'BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD', 'SOL-USD', 
            'USDC-USD', 'XRP-USD', 'DOGE-USD', 'TON11419-USD', 'ADA-USD',
            'SHIB-USD', 'AVAX-USD', 'TRX-USD', 'DOT-USD', 'BCH-USD',
            'NEAR-USD', 'MATIC-USD', 'ICP-USD', 'UNI7083-USD', 'LTC-USD',
            'DAI-USD', 'LEO-USD', 'ETC-USD', 'APT21794-USD', 'STX4847-USD',
            'CRO-USD', 'OKB-USD', 'ATOM-USD', 'FIL-USD', 'IMX10603-USD',
            'VET-USD', 'MNT27075-USD', 'ARB11841-USD', 'HBAR-USD', 'OP-USD',
            'MKR-USD', 'AAVE-USD', 'GRT6719-USD', 'SEI23149-USD', 'SUI20947-USD',
            'THETA-USD', 'RUNE-USD', 'FTM-USD', 'ALGO-USD', 'FLOW-USD',
            'SAND-USD', 'XLM-USD', 'AXS-USD', 'MANA-USD', 'CHZ-USD',
            'EGLD-USD', 'KCS-USD', 'XTZ-USD', 'EOS-USD', 'CAKE-USD',
            'QNT-USD', 'ASTR-USD', 'FEI-USD', 'KLAY-USD', 'NEO-USD',
            'IOTA-USD', 'BSV-USD', 'XMR-USD', 'COMP-USD', 'ZEC-USD',
            'DASH-USD', 'WAVES-USD', 'QTUM-USD', 'BAT-USD', 'DYDX-USD',
            'CRV-USD', 'ENJ-USD', '1INCH-USD', 'ZIL-USD', 'SUSHI-USD',
            'YFI-USD', 'REN-USD', 'BNT-USD', 'SNX-USD', 'UMA-USD',
            'STORJ-USD', 'BAL-USD', 'NMR-USD', 'LRC-USD', 'KNC-USD',
            'BAND-USD', 'RSR-USD', 'RLC-USD', 'REP-USD', 'ZRX-USD',
            'OXT-USD', 'MLN-USD', 'FARM-USD', 'BADGER-USD', 'PICKLE-USD',
            'ALPHA-USD', 'CREAM-USD', 'TORN-USD', 'COVER-USD', 'INDEX-USD',
            'DPI-USD', 'PERP-USD', 'API3-USD', 'KEEP-USD', 'NU-USD',
            'MASK-USD', 'TRU-USD', 'RAD-USD', 'CTX-USD', 'BOND-USD',
            'POOL-USD', 'FOX-USD', 'TRIBE-USD', 'RALLY-USD', 'XYO-USD',
            'REQ-USD', 'POLY-USD', 'PRQ-USD', 'REEF-USD', 'POLS-USD',
            'OGN-USD', 'NKN-USD', 'LPT-USD', 'RGT-USD', 'PPAY-USD',
            'ANKR-USD', 'CVC-USD', 'GNO-USD', 'MPH-USD', 'RARI-USD',
            'TOKE-USD', 'ALCX-USD', 'FXS-USD', 'CVX-USD', 'SPELL-USD',
            'JPEG-USD', 'TRIBE-USD', 'VISR-USD', 'TRAC-USD', 'QSP-USD',
            'RBN-USD', 'OCEAN-USD', 'FET-USD', 'AGIX-USD', 'RNDR-USD',
            'INJ-USD', 'CFG-USD', 'COTI-USD', 'ERG-USD', 'CKB-USD'
        ]
        
        # 시가총액 정보를 가져와서 정렬
        valid_symbols = []
        market_caps = {}
        
        print("📊 각 코인의 시가총액 정보 수집 중...")
        for symbol in tqdm(major_cryptos[:200], desc="시가총액 조회"):
            try:
                ticker = yf.Ticker(symbol)
                info = ticker.info
                market_cap = info.get('marketCap', 0)
                
                if market_cap and market_cap > 0:
                    market_caps[symbol] = market_cap
                    valid_symbols.append(symbol)
                
                time.sleep(0.05)
                
            except Exception as e:
                continue
        
        # 시가총액 순으로 정렬하여 상위 N개 선택
        sorted_symbols = sorted(valid_symbols, key=lambda x: market_caps.get(x, 0), reverse=True)
        self.symbols = sorted_symbols[:self.top_n]
        
        print(f"✅ 상위 {len(self.symbols)}개 코인 티커 선정 완료")
        print(f"📈 Top 10 코인:")
        for i, symbol in enumerate(self.symbols[:10]):
            market_cap_b = market_caps[symbol] / 1e9
            print(f"   {i+1:2d}. {symbol:<12} (시총: ${market_cap_b:.1f}B)")
        
        return self.symbols
        
    def download_basic_data(self, chunk_size=10):  # 1시간봉의 경우 chunk_size를 줄임
        """기본 OHLCV 1시간봉 데이터 다운로드"""
        print("📈 기본 OHLCV 1시간봉 데이터 다운로드 중...")
        print("⚠️ 주의: 1시간봉 데이터는 야후 파이낸스에서 제한된 기간만 제공됩니다.")
        
        # 1시간봉의 경우 개별적으로 다운로드하는 것이 더 안정적
        for symbol in tqdm(self.symbols, desc="1시간봉 데이터"):
            try:
                # 1시간봉 데이터 다운로드
                data = yf.download(
                    symbol, 
                    start=self.start_date, 
                    end=self.end_date,
                    interval='1h',  # 1시간봉 설정
                    progress=False,
                    auto_adjust=True,
                    prepost=False
                )
                
                # 데이터 정리: 기본 OHLCV 컬럼만 유지
                if not data.empty:
                    # 컬럼명 표준화
                    required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
                    available_columns = [col for col in required_columns if col in data.columns]
                    
                    if len(available_columns) >= 4:  # OHLC는 최소한 있어야 함
                        clean_data = data[available_columns].copy()
                        clean_data = clean_data.dropna(how='all')  # 모든 값이 NaN인 행 제거
                        
                        if len(clean_data) > 0:
                            self.crypto_data[symbol] = clean_data
                            print(f"✅ {symbol}: {len(clean_data)}개 1시간봉 데이터 수집 완료")
                        else:
                            self.failed_symbols.append(symbol)
                            print(f"❌ {symbol}: 유효한 데이터 없음")
                    else:
                        self.failed_symbols.append(symbol)
                        print(f"❌ {symbol}: 필수 컬럼 부족")
                else:
                    self.failed_symbols.append(symbol)
                    print(f"❌ {symbol}: 빈 데이터")
                    
            except Exception as e:
                self.failed_symbols.append(symbol)
                print(f"⚠️ {symbol} 1시간봉 데이터 다운로드 실패: {e}")
            
            time.sleep(0.5)  # API 호출 간격 조정
    
    def save_basic_data(self):
        """기본 OHLCV 1시간봉 데이터를 CSV로 저장"""
        print("💾 1시간봉 데이터 CSV 파일 저장 중...")
        
        # 출력 디렉토리 생성
        os.makedirs(self.output, exist_ok=True)
        os.makedirs(f"{self.output}/raw_data_1h", exist_ok=True)
        
        # 개별 코인 CSV 저장
        saved_count = 0
        for symbol, data in tqdm(self.crypto_data.items(), desc="CSV 저장"):
            if not data.empty:
                # 파일명에서 특수문자 제거
                safe_symbol = symbol.replace('-USD', '').replace('/', '_')
                filename = f"{self.output}/raw_data_1h/{safe_symbol}_1h.csv"
                
                # Date 컬럼 추가 (인덱스를 컬럼으로)
                data_to_save = data.copy()
                data_to_save.reset_index(inplace=True)
                data_to_save.rename(columns={'Datetime': 'Datetime'}, inplace=True)
                
                data_to_save.to_csv(filename, index=False)
                saved_count += 1
        
        # 다운로드 요약 정보
        summary = {
            'total_symbols': len(self.symbols),
            'successful_downloads': saved_count,
            'failed_downloads': len(self.failed_symbols),
            'failed_symbols': self.failed_symbols,
            'download_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'date_range': f"{self.start_date} to {self.end_date}",
            'data_type': 'Hourly OHLCV (1h interval)',
            'interval': '1h'
        }
        
        summary_df = pd.DataFrame([summary])
        summary_df.to_csv(f"{self.output}/download_summary_1h.csv", index=False)
        
        # 성공한 심볼 리스트 저장
        if saved_count > 0:
            success_symbols = [symbol for symbol in self.crypto_data.keys() if not self.crypto_data[symbol].empty]
            symbols_df = pd.DataFrame({'Symbol': success_symbols})
            symbols_df.to_csv(f"{self.output}/successful_symbols_1h.csv", index=False)
        
        print(f"✅ 1시간봉 데이터 저장 완료!")
        print(f"📁 저장 위치: {self.output}/raw_data_1h/")
        print(f"✅ 성공: {saved_count}개")
        print(f"❌ 실패: {len(self.failed_symbols)}개")
        if self.failed_symbols:
            print(f"실패 종목: {', '.join(self.failed_symbols[:10])}{'...' if len(self.failed_symbols) > 10 else ''}")
    
    def run_download(self):
        """1시간봉 데이터 다운로드 프로세스 실행"""
        print("🚀 암호화폐 1시간봉 데이터 다운로드 시작!")
        print(f"🪙 대상: 시가총액 상위 {self.top_n}개 코인")
        print(f"📅 기간: {self.start_date} ~ {self.end_date}")
        print("⏰ 간격: 1시간봉")
        print("="*50)
        
        # 1. 상위 코인 티커 가져오기
        self.get_top_crypto_symbols()
        
        # 2. 1시간봉 데이터 다운로드
        self.download_basic_data()
        
        # 3. CSV 저장
        self.save_basic_data()
        
        print("="*50)
        print("🎉 1시간봉 데이터 다운로드 완료!")
        
        return self.crypto_data


In [16]:
start_date = "2023-10-01" # 1시간봉 데이터는 최근 730일(약 2년)만 제공
end_date="2025-09-21"

In [ ]:


# 실행
if __name__ == "__main__":
    # 다운로더 인스턴스 생성
    downloader = SimpleCryptoDownloader(
        start_date=start_date,  # 1시간봉 데이터는 최근 730일(약 2년)만 제공
        end_date=end_date,
        output='/workspace/AI모델/projects/coin/data/1h',
        top_n=20  # 상위 100개만
    )
    
    # 1시간봉 데이터 다운로드 실행
    crypto_data = downloader.run_download()
    
    print(f"\n📋 다운로드된 코인 수: {len(crypto_data)}")
    if crypto_data:
        sample_symbol = list(crypto_data.keys())[0]
        sample_data = crypto_data[sample_symbol]
        print(f"🪙 샘플 데이터 ({sample_symbol}):")
        print(f"   - 기간: {sample_data.index[0]} ~ {sample_data.index[-1]}")
        print(f"   - 행 수: {len(sample_data)}")
        print(f"   - 컬럼: {list(sample_data.columns)}")
        print(f"   - 첫 3행:\n{sample_data.head(3)}")

🚀 암호화폐 1시간봉 데이터 다운로드 시작!
🪙 대상: 시가총액 상위 20개 코인
📅 기간: 2023-10-01 ~ 2025-09-21
⏰ 간격: 1시간봉
🔍 시가총액 기준 상위 20개 코인 티커 검색 중...
📊 각 코인의 시가총액 정보 수집 중...


시가총액 조회: 100%|██████████| 150/150 [01:22<00:00,  1.83it/s]


✅ 상위 20개 코인 티커 선정 완료
📈 Top 10 코인:
    1. BTC-USD      (시총: $2241.4B)
    2. ETH-USD      (시총: $505.7B)
    3. USDT-USD     (시총: $172.1B)
    4. XRP-USD      (시총: $168.3B)
    5. BNB-USD      (시총: $142.9B)
    6. SOL-USD      (시총: $119.8B)
    7. USDC-USD     (시총: $73.9B)
    8. DOGE-USD     (시총: $36.1B)
    9. TRX-USD      (시총: $31.7B)
   10. ADA-USD      (시총: $29.4B)
📈 기본 OHLCV 1시간봉 데이터 다운로드 중...
⚠️ 주의: 1시간봉 데이터는 야후 파이낸스에서 제한된 기간만 제공됩니다.


1시간봉 데이터:   0%|          | 0/20 [00:00<?, ?it/s]

✅ BTC-USD: 17302개 1시간봉 데이터 수집 완료


...

✅ CRO-USD: 17293개 1시간봉 데이터 수집 완료


1시간봉 데이터: 100%|██████████| 20/20 [00:31<00:00,  1.57s/it]


💾 1시간봉 데이터 CSV 파일 저장 중...


CSV 저장: 100%|██████████| 20/20 [00:03<00:00,  5.55it/s]


✅ 1시간봉 데이터 저장 완료!
📁 저장 위치: /workspace/AI모델/projects/coin/data/1h/raw_data_1h/
✅ 성공: 20개
❌ 실패: 0개
🎉 1시간봉 데이터 다운로드 완료!

📋 다운로드된 코인 수: 20
🪙 샘플 데이터 (BTC-USD):
   - 기간: 2023-10-01 00:00:00+00:00 ~ 2025-09-20 23:00:00+00:00
   - 행 수: 17302
   - 컬럼: [('Open', 'BTC-USD'), ('High', 'BTC-USD'), ('Low', 'BTC-USD'), ('Close', 'BTC-USD'), ('Volume', 'BTC-USD')]
   - 첫 3행:
Price                              Open          High           Low  \
Ticker                          BTC-USD       BTC-USD       BTC-USD   
Datetime                                                              
2023-10-01 00:00:00+00:00  26967.396484  26999.722656  26965.093750   
2023-10-01 01:00:00+00:00  26995.111328  27032.787109  26988.476562   
2023-10-01 02:00:00+00:00  27027.078125  27048.111328  27026.638672   

Price                             Close    Volume  
Ticker                          BTC-USD   BTC-USD  
Datetime                                           
2023-10-01 00:00:00+00:00  26995.140625         0  
202

# 지표코드

In [21]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

class CryptoTechnicalIndicators:
    def __init__(self, input_folder, output_folder, start_date=None, end_date=None):
        """
        기술지표 계산기 초기화
        
        Parameters:
        input_folder: 원본 CSV 파일들이 있는 폴더 경로
        output_folder: 기술지표가 추가된 CSV를 저장할 폴더 경로
        start_date: 계산 시작 날짜 (없으면 전체 기간)
        end_date: 계산 종료 날짜 (없으면 전체 기간)
        """
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.start_date = start_date
        self.end_date = end_date
        self.processed_data = {}
        self.failed_symbols = []
        
    def load_crypto_data(self, symbol_file):
        """개별 코인 CSV 파일 로드"""
        try:
            df = pd.read_csv(symbol_file)
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.get_level_values(0)

            # Date 컬럼을 datetime으로 변환하고 인덱스로 설정
            if 'Datetime' in df.columns:
                df['Datetime'] = pd.to_datetime(df['Datetime'])
                df.set_index('Datetime', inplace=True)
            elif 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'])
                df.set_index('Date', inplace=True)
            
            # 타임존 정보 제거
            if hasattr(df.index, 'tz') and df.index.tz is not None:
                df.index = df.index.tz_localize(None)
            
            # **중요: 수치 컬럼들을 숫자로 변환**
            numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # 기간 필터링
            if self.start_date:
                df = df[df.index >= self.start_date]
            if self.end_date:
                df = df[df.index <= self.end_date]
            
            # 필수 컬럼 확인
            required_columns = ['Open', 'High', 'Low', 'Close']
            if not all(col in df.columns for col in required_columns):
                return None
            
            # 데이터 정렬 (날짜순)
            df = df.sort_index()
            
            return df
            
        except Exception as e:
            print(f"⚠️ 파일 로드 실패 {symbol_file}: {e}")
            return None
    
    def calculate_moving_averages(self, df):
        """이동평균 계산 - 전략에 필요한 추가 기간 포함"""
        try:
            # 단순 이동평균 (전략에서 요구하는 모든 기간)
            periods = [2, 3, 4, 5, 6, 7, 8, 10, 13, 16, 20, 25, 29, 50, 60, 100, 120, 200]
            for period in periods:
                if len(df) >= period:
                    df[f'MA_{period}'] = df['Close'].rolling(window=period, min_periods=period).mean()
            
            # 지수이동평균 (전략 필요 기간)
            ema_periods = [3, 4, 5, 6, 7, 10, 12, 20, 24, 26, 50]
            for period in ema_periods:
                if len(df) >= period:
                    df[f'EMA_{period}'] = df['Close'].ewm(span=period, min_periods=period).mean()
            
        except Exception as e:
            print(f"⚠️ 이동평균 계산 실패: {e}")
        
        return df
        
    def calculate_macd_variations(self, df):
        """다양한 MACD 변형 계산"""
        try:
            # 기본 MACD (12,26,9)
            if 'EMA_12' in df.columns and 'EMA_26' in df.columns:
                df['MACD'] = df['EMA_12'] - df['EMA_26']
                df['MACD_Signal'] = df['MACD'].ewm(span=9, min_periods=9).mean()
                df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
            
            # MACD (10,20,9) - 메모 1-58번용
            if len(df) >= 20:
                ema_10 = df['Close'].ewm(span=10, min_periods=10).mean()
                ema_20_macd = df['Close'].ewm(span=20, min_periods=20).mean()
                df['MACD_10_20'] = ema_10 - ema_20_macd
                df['MACD_10_20_Signal'] = df['MACD_10_20'].ewm(span=9, min_periods=9).mean()
            
            # MACD (15,26,9) - 메모 1-144번용
            if len(df) >= 26:
                ema_15 = df['Close'].ewm(span=15, min_periods=15).mean()
                if 'EMA_26' in df.columns:
                    df['MACD_15_26'] = ema_15 - df['EMA_26']
                    df['MACD_15_26_Signal'] = df['MACD_15_26'].ewm(span=9, min_periods=9).mean()
            
            # MACD (5,27,9) - 메모 3-292번용
            if len(df) >= 27:
                ema_5_macd = df['Close'].ewm(span=5, min_periods=5).mean()
                ema_27 = df['Close'].ewm(span=27, min_periods=27).mean()
                df['MACD_5_27'] = ema_5_macd - ema_27
                df['MACD_5_27_Signal'] = df['MACD_5_27'].ewm(span=9, min_periods=9).mean()
                
        except Exception as e:
            print(f"⚠️ MACD 변형 계산 실패: {e}")
        
        return df
    
    def calculate_rsi_variations(self, df):
        """다양한 RSI 기간 계산"""
        try:
            # 전략에서 필요한 모든 RSI 기간
            rsi_periods = [4, 5, 6, 7, 12, 14, 20, 21]
            
            for period in rsi_periods:
                if len(df) >= period + 1:
                    delta = df['Close'].diff()
                    gain = (delta.where(delta > 0, 0)).rolling(window=period, min_periods=period).mean()
                    loss = (-delta.where(delta < 0, 0)).rolling(window=period, min_periods=period).mean()
                    
                    rs = np.where(loss > 0, gain / loss, 0)
                    df[f'RSI_{period}'] = 100 - (100 / (1 + rs))
                    
        except Exception as e:
            print(f"⚠️ RSI 변형 계산 실패: {e}")
        
        return df
    
    def calculate_bollinger_bands(self, df, period=20, std_mult=2):
        """볼린저 밴드 계산"""
        try:
            if len(df) >= period:
                df[f'BB_Middle_{period}'] = df['Close'].rolling(window=period, min_periods=period).mean()
                bb_std = df['Close'].rolling(window=period, min_periods=period).std()
                df[f'BB_Upper_{period}'] = df[f'BB_Middle_{period}'] + (bb_std * std_mult)
                df[f'BB_Lower_{period}'] = df[f'BB_Middle_{period}'] - (bb_std * std_mult)
                df[f'BB_Width_{period}'] = df[f'BB_Upper_{period}'] - df[f'BB_Lower_{period}']
                
                # BB Position (0~1 사이의 값)
                df[f'BB_Position_{period}'] = np.where(
                    df[f'BB_Width_{period}'] > 0,
                    (df['Close'] - df[f'BB_Lower_{period}']) / df[f'BB_Width_{period}'],
                    0.5
                )
        except Exception as e:
            print(f"⚠️ 볼린저 밴드 계산 실패: {e}")
        
        return df
    
    def calculate_returns(self, df):
        """수익률 계산"""
        try:
            # 다양한 기간의 수익률
            periods = [1, 3, 7, 14, 30, 90, 180, 365]
            
            # 일일 수익률
            df['Return_1d'] = df['Close'].pct_change(periods=1)
            
            # 다기간 수익률
            for period in periods[1:]:  # 1일 제외
                if len(df) > period:
                    df[f'Return_{period}d'] = df['Close'] / df['Close'].shift(period) - 1
            
            # 누적 수익률
            df['Cumulative_Return'] = (1 + df['Return_1d'].fillna(0)).cumprod() - 1
            
        except Exception as e:
            print(f"⚠️ 수익률 계산 실패: {e}")
        
        return df
    
    def calculate_volatility(self, df):
        """변동성 계산"""
        try:
            periods = [7, 14, 30, 90]
            
            for period in periods:
                if len(df) >= period and 'Return_1d' in df.columns:
                    df[f'Volatility_{period}d'] = df['Return_1d'].rolling(
                        window=period, min_periods=period
                    ).std() * np.sqrt(365)  # 연간화
                    
        except Exception as e:
            print(f"⚠️ 변동성 계산 실패: {e}")
        
        return df
    
    def calculate_volume_indicators(self, df):
        """거래량 지표 계산"""
        try:
            if 'Volume' in df.columns:
                df['Volume'] = df['Volume'].replace(0, np.nan).fillna(method='ffill')
                # 거래량 이동평균
                periods = [7, 20, 50]
                for period in periods:
                    if len(df) >= period:
                        df[f'Volume_MA_{period}'] = df['Volume'].rolling(
                            window=period, min_periods=period
                        ).mean()
                
                # 거래량 비율
                if 'Volume_MA_20' in df.columns:
                    df['Volume_Ratio'] = np.where(
                        df['Volume_MA_20'] > 0,
                        df['Volume'] / df['Volume_MA_20'],
                        1
                    )
                
                # 거래량-가격 트렌드
                df['Volume_Price_Trend'] = df['Volume'] * df['Close']
                
        except Exception as e:
            print(f"⚠️ 거래량 지표 계산 실패: {e}")
        
        return df
    
    def calculate_atr(self, df, period=14):
        """ATR (Average True Range) 계산"""
        try:
            if len(df) >= 2:
                # True Range 계산
                df['True_Range'] = np.maximum(
                    df['High'] - df['Low'],
                    np.maximum(
                        abs(df['High'] - df['Close'].shift(1)),
                        abs(df['Low'] - df['Close'].shift(1))
                    )
                )
                
                if len(df) >= period:
                    df[f'ATR_{period}'] = df['True_Range'].rolling(
                        window=period, min_periods=period
                    ).mean()
                    
                    # ATR 퍼센트
                    df[f'ATR_Percent_{period}'] = np.where(
                        df['Close'] > 0,
                        df[f'ATR_{period}'] / df['Close'] * 100,
                        0
                    )
                    
        except Exception as e:
            print(f"⚠️ ATR 계산 실패: {e}")
        
        return df
    
    def calculate_stochastic_variations(self, df):
        """다양한 스토캐스틱 기간 계산"""
        try:
            # 전략에서 필요한 스토캐스틱 조합들
            stoch_configs = [
                (1, 3),    # 메모 8-432, 10-120번용
                (3, 3),    # 메모 3-231번용
                (4, 2),    # 메모 2-249번용
                (5, 3),    # 메모 7-248번용
                (6, 2),    # 메모 7-369번용
                (14, 3)    # 기본
            ]
            
            for k_period, d_period in stoch_configs:
                if len(df) >= k_period:
                    low_k = df['Low'].rolling(window=k_period, min_periods=k_period).min()
                    high_k = df['High'].rolling(window=k_period, min_periods=k_period).max()
                    range_k = high_k - low_k
                    
                    df[f'Stoch_K_{k_period}'] = np.where(
                        range_k > 0,
                        100 * ((df['Close'] - low_k) / range_k),
                        50
                    )
                    
                    if len(df) >= k_period + d_period - 1:
                        df[f'Stoch_D_{k_period}'] = df[f'Stoch_K_{k_period}'].rolling(
                            window=d_period, min_periods=d_period
                        ).mean()
                        
        except Exception as e:
            print(f"⚠️ 스토캐스틱 변형 계산 실패: {e}")
        
        return df

    def calculate_williams_r_variations(self, df):
        """다양한 Williams %R 기간 계산"""
        try:
            # 전략에서 필요한 Williams %R 기간들
            wr_periods = [1, 8, 10, 14]
            
            for period in wr_periods:
                if len(df) >= period:
                    low_period = df['Low'].rolling(window=period, min_periods=period).min()
                    high_period = df['High'].rolling(window=period, min_periods=period).max()
                    range_period = high_period - low_period
                    
                    df[f'Williams_R_{period}'] = np.where(
                        range_period > 0,
                        -100 * (high_period - df['Close']) / range_period,
                        -50
                    )
            
            # 기본 Williams %R도 유지 (14일 기준)
            if 'Williams_R_14' in df.columns:
                df['Williams_R'] = df['Williams_R_14']
                    
        except Exception as e:
            print(f"⚠️ Williams %R 변형 계산 실패: {e}")
        
        return df

    def calculate_cci_variations(self, df):
        """다양한 CCI 기간 계산"""
        try:
            # 전략에서 필요한 CCI 기간들
            cci_periods = [3, 4, 10, 17, 20]
            
            for period in cci_periods:
                if len(df) >= period:
                    tp = (df['High'] + df['Low'] + df['Close']) / 3
                    cci_ma = tp.rolling(window=period, min_periods=period).mean()
                    cci_mad = tp.rolling(window=period, min_periods=period).apply(
                        lambda x: np.mean(np.abs(x - np.mean(x))), raw=True
                    )
                    
                    df[f'CCI_{period}'] = np.where(
                        cci_mad > 0,
                        (tp - cci_ma) / (0.015 * cci_mad),
                        0
                    )
            
            # 기본 CCI도 유지 (20일 기준)
            if 'CCI_20' in df.columns:
                df['CCI'] = df['CCI_20']
                
        except Exception as e:
            print(f"⚠️ CCI 변형 계산 실패: {e}")
        
        return df

    def calculate_mfi_variations(self, df):
        """다양한 MFI 기간 계산"""
        try:
            if 'Volume' not in df.columns:
                return df
                
            # 전략에서 필요한 MFI 기간들
            mfi_periods = [11, 14]
            
            for period in mfi_periods:
                if len(df) >= period:
                    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
                    money_flow = typical_price * df['Volume']
                    mf_sign = np.where(typical_price > typical_price.shift(1), 1, -1)
                    mf_signed = money_flow * mf_sign
                    
                    mf_pos = mf_signed.where(mf_signed > 0, 0).rolling(period, min_periods=period).sum()
                    mf_neg = -mf_signed.where(mf_signed < 0, 0).rolling(period, min_periods=period).sum()
                    
                    df[f'MFI_{period}'] = np.where(
                        mf_neg > 0,
                        100 - (100 / (1 + mf_pos / mf_neg)),
                        50
                    )
            
            # 기본 MFI도 유지 (14일 기준)
            if 'MFI_14' in df.columns:
                df['MFI'] = df['MFI_14']
                
        except Exception as e:
            print(f"⚠️ MFI 변형 계산 실패: {e}")
        
        return df

    def calculate_price_oscillator(self, df):
        """Price Oscillator 계산 (5차 전략용)"""
        try:
            # 전략에서 필요한 Price Oscillator 조합들
            oscp_configs = [
                (11, 36),  # 메모 5차 전략용
                (18, 19),  # 메모 5차 전략용  
                (19, 20)   # 메모 5차 전략용
            ]
            
            for short, long in oscp_configs:
                if len(df) >= long:
                    short_ma = df['Close'].rolling(window=short, min_periods=short).mean()
                    long_ma = df['Close'].rolling(window=long, min_periods=long).mean()
                    
                    df[f'OSCP_{short}_{long}'] = np.where(
                        long_ma > 0,
                        ((short_ma - long_ma) / long_ma) * 100,
                        0
                    )
                    
        except Exception as e:
            print(f"⚠️ Price Oscillator 계산 실패: {e}")
        
        return df

    def calculate_dmi_adx(self, df):
        """DMI와 ADX 계산"""
        try:
            period = 21  # 메모 1-116번용
            
            if len(df) >= period + 1:
                # True Range 계산
                df['TR'] = np.maximum(
                    df['High'] - df['Low'],
                    np.maximum(
                        abs(df['High'] - df['Close'].shift(1)),
                        abs(df['Low'] - df['Close'].shift(1))
                    )
                )
                
                # Directional Movement 계산
                df['DM_Plus'] = np.where(
                    (df['High'] - df['High'].shift(1)) > (df['Low'].shift(1) - df['Low']),
                    np.maximum(df['High'] - df['High'].shift(1), 0),
                    0
                )
                
                df['DM_Minus'] = np.where(
                    (df['Low'].shift(1) - df['Low']) > (df['High'] - df['High'].shift(1)),
                    np.maximum(df['Low'].shift(1) - df['Low'], 0),
                    0
                )
                
                # Smoothed values
                tr_smooth = df['TR'].rolling(window=period, min_periods=period).mean()
                dm_plus_smooth = df['DM_Plus'].rolling(window=period, min_periods=period).mean()
                dm_minus_smooth = df['DM_Minus'].rolling(window=period, min_periods=period).mean()
                
                # DI+ and DI-
                df[f'DI_Plus_{period}'] = np.where(tr_smooth > 0, 100 * dm_plus_smooth / tr_smooth, 0)
                df[f'DI_Minus_{period}'] = np.where(tr_smooth > 0, 100 * dm_minus_smooth / tr_smooth, 0)
                
                # ADX
                dx = np.where(
                    (df[f'DI_Plus_{period}'] + df[f'DI_Minus_{period}']) > 0,
                    100 * abs(df[f'DI_Plus_{period}'] - df[f'DI_Minus_{period}']) / (df[f'DI_Plus_{period}'] + df[f'DI_Minus_{period}']),
                    0
                )
                df[f'ADX_{period}'] = pd.Series(dx).rolling(window=period, min_periods=period).mean()
                
        except Exception as e:
            print(f"⚠️ DMI/ADX 계산 실패: {e}")
        
        return df

    def calculate_price_roc(self, df):
        """Price Rate of Change 계산"""
        try:
            # 전략에서 필요한 ROC 기간들
            roc_periods = [3]  # 메모 10-540번용
            
            for period in roc_periods:
                if len(df) >= period + 1:
                    df[f'Price_ROC_{period}'] = ((df['Close'] / df['Close'].shift(period)) - 1) * 100
                    
        except Exception as e:
            print(f"⚠️ Price ROC 계산 실패: {e}")
        
        return df

    def calculate_standard_deviation(self, df):
        """표준편차 계산 (5차 전략용)"""
        try:
            periods = [1, 20]  # 전략에서 사용하는 기간들
            
            for period in periods:
                if len(df) >= period:
                    df[f'STD_{period}'] = df['Close'].rolling(window=period, min_periods=period).std()
                    
        except Exception as e:
            print(f"⚠️ 표준편차 계산 실패: {e}")
        
        return df

    def calculate_highest_lowest(self, df):
        """최고가/최저가 계산 (전략용)"""
        try:
            # 전략에서 필요한 기간들
            periods = [3, 7, 8, 10, 11, 16, 17, 28]
            
            for period in periods:
                if len(df) >= period:
                    df[f'Highest_{period}'] = df['Close'].rolling(window=period, min_periods=period).max()
                    df[f'Lowest_{period}'] = df['Close'].rolling(window=period, min_periods=period).min()
                    
        except Exception as e:
            print(f"⚠️ 최고가/최저가 계산 실패: {e}")
        
        return df

    def calculate_advanced_indicators(self, df):
        """고급 지표들 계산 - 통합된 함수"""
        try:
            # 기존 지표들 유지 (기본 Williams %R, CCI, MFI)
            
            # Williams %R (기본 14일)
            if len(df) >= 14:
                low_14 = df['Low'].rolling(window=14, min_periods=14).min()
                high_14 = df['High'].rolling(window=14, min_periods=14).max()
                range_14 = high_14 - low_14
                
                df['Williams_R'] = np.where(
                    range_14 > 0,
                    -100 * (high_14 - df['Close']) / range_14,
                    -50
                )
            
            # CCI (기본 20일)
            if len(df) >= 20:
                tp = (df['High'] + df['Low'] + df['Close']) / 3
                cci_ma = tp.rolling(window=20, min_periods=20).mean()
                cci_mad = tp.rolling(window=20, min_periods=20).apply(
                    lambda x: np.mean(np.abs(x - np.mean(x))), raw=True
                )
                
                df['CCI'] = np.where(
                    cci_mad > 0,
                    (tp - cci_ma) / (0.015 * cci_mad),
                    0
                )
            
            # MFI (기본 14일)
            if len(df) >= 14 and 'Volume' in df.columns:
                typical_price = (df['High'] + df['Low'] + df['Close']) / 3
                money_flow = typical_price * df['Volume']
                mf_sign = np.where(typical_price > typical_price.shift(1), 1, -1)
                mf_signed = money_flow * mf_sign
                
                mf_pos = mf_signed.where(mf_signed > 0, 0).rolling(14, min_periods=14).sum()
                mf_neg = -mf_signed.where(mf_signed < 0, 0).rolling(14, min_periods=14).sum()
                
                df['MFI'] = np.where(
                    mf_neg > 0,
                    100 - (100 / (1 + mf_pos / mf_neg)),
                    50
                )
                
        except Exception as e:
            print(f"⚠️ 고급지표 계산 실패: {e}")
        
        return df
    
    def calculate_price_ratios(self, df):
        """가격 비율 지표 계산"""
        try:
            # 이동평균 대비 가격 비율
            ma_periods = [7, 20, 50, 100, 200]
            for period in ma_periods:
                ma_col = f'MA_{period}'
                if ma_col in df.columns:
                    df[f'Price_vs_MA{period}'] = np.where(
                        df[ma_col] > 0,
                        df['Close'] / df[ma_col] - 1,
                        0
                    )
            
            # 고점/저점 대비 현재가 위치
            periods = [52, 200]  # 52일, 200일
            for period in periods:
                if len(df) >= period:
                    rolling_high = df['High'].rolling(window=period, min_periods=period).max()
                    rolling_low = df['Low'].rolling(window=period, min_periods=period).min()
                    range_hl = rolling_high - rolling_low
                    
                    df[f'HighLow_Position_{period}d'] = np.where(
                        range_hl > 0,
                        (df['Close'] - rolling_low) / range_hl,
                        0.5
                    )
                    
        except Exception as e:
            print(f"⚠️ 가격비율 계산 실패: {e}")
        
        return df
    
    def process_single_crypto(self, csv_file):
            """개별 코인 파일 처리 - 향상된 버전"""
            symbol = os.path.basename(csv_file).replace('.csv', '')
            
            try:
                print(f"📊 {symbol} 처리 중...")
                
                # 1. 데이터 로드
                df = self.load_crypto_data(csv_file)
                if df is None or len(df) < 30:  # 최소 데이터 요구량 증가
                    self.failed_symbols.append(symbol)
                    return None
                
                print(f"   - 데이터 기간: {df.index[0]} ~ {df.index[-1]} ({len(df)}일)")
                
                # 2. 기본 기술지표 계산
                df = self.calculate_moving_averages(df)
                df = self.calculate_macd_variations(df)  # 수정된 함수
                df = self.calculate_rsi_variations(df)   # 수정된 함수
                df = self.calculate_bollinger_bands(df)
                df = self.calculate_returns(df)
                df = self.calculate_volatility(df)
                df = self.calculate_volume_indicators(df)
                df = self.calculate_atr(df)
                
                # 3. 전략용 추가 지표 계산
                df = self.calculate_stochastic_variations(df)  # 새로 추가
                df = self.calculate_williams_r_variations(df)  # 새로 추가
                df = self.calculate_cci_variations(df)         # 새로 추가
                df = self.calculate_mfi_variations(df)         # 새로 추가
                df = self.calculate_price_oscillator(df)       # 새로 추가
                df = self.calculate_dmi_adx(df)               # 새로 추가
                df = self.calculate_price_roc(df)             # 새로 추가
                df = self.calculate_standard_deviation(df)     # 새로 추가
                df = self.calculate_highest_lowest(df)        # 새로 추가
                df = self.calculate_advanced_indicators(df)
                df = self.calculate_price_ratios(df)
                
                # 4. 무한값, NaN 처리
                df = df.replace([np.inf, -np.inf], np.nan)
                
                # 5. 결과 저장
                self.processed_data[symbol] = df
                
                print(f"   - 최종 컬럼 수: {len(df.columns)}")
                
                return df
                
            except Exception as e:
                print(f"⚠️ {symbol} 처리 실패: {e}")
                self.failed_symbols.append(symbol)
                return None
    
    def process_all_cryptos(self):
        """모든 코인 파일 처리"""
        print("🚀 기술지표 계산 시작!")
        print(f"📁 입력 폴더: {self.input_folder}")
        print(f"📅 기간: {self.start_date or '전체'} ~ {self.end_date or '전체'}")
        print("="*50)
        
        # CSV 파일 목록 가져오기
        csv_files = [f for f in os.listdir(self.input_folder) if f.endswith('.csv')]
        
        if not csv_files:
            print("❌ CSV 파일을 찾을 수 없습니다!")
            return
        
        print(f"📊 총 {len(csv_files)}개 파일 발견")
        
        # 각 파일 처리
        for csv_file in tqdm(csv_files, desc="기술지표 계산"):
            file_path = os.path.join(self.input_folder, csv_file)
            self.process_single_crypto(file_path)
        
        print("="*50)
        print(f"✅ 성공: {len(self.processed_data)}개")
        print(f"❌ 실패: {len(self.failed_symbols)}개")
        if self.failed_symbols:
            print(f"실패 종목: {', '.join(self.failed_symbols[:10])}{'...' if len(self.failed_symbols) > 10 else ''}")
    
    def save_processed_data(self):
        """처리된 데이터 저장"""
        print("💾 기술지표 추가된 데이터 저장 중...")
        
        # 출력 폴더 생성
        os.makedirs(self.output_folder, exist_ok=True)
        os.makedirs(f"{self.output_folder}/with_indicators", exist_ok=True)
        
        saved_count = 0
        
        # 개별 파일 저장
        for symbol, df in tqdm(self.processed_data.items(), desc="저장"):
            try:
                # Date를 첫 번째 컬럼으로 복원
                df_to_save = df.copy()
                df_to_save.reset_index(inplace=True)
                
                filename = f"{self.output_folder}/with_indicators/{symbol}.csv"
                df_to_save.to_csv(filename, index=False)
                saved_count += 1
                
            except Exception as e:
                print(f"⚠️ {symbol} 저장 실패: {e}")
        
        # 통합 데이터셋 생성 (선택적)
        if self.processed_data:
            self.create_combined_dataset()
        
        # 처리 요약 저장
        summary = {
            'total_processed': len(self.processed_data),
            'total_failed': len(self.failed_symbols),
            'failed_symbols': self.failed_symbols,
            'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'period_filter': f"{self.start_date or 'None'} to {self.end_date or 'None'}"
        }
        
        summary_df = pd.DataFrame([summary])
        summary_df.to_csv(f"{self.output_folder}/processing_summary.csv", index=False)
        
        print(f"✅ 기술지표 데이터 저장 완료!")
        print(f"📁 저장 위치: {self.output_folder}/with_indicators/")
        print(f"✅ 저장 완료: {saved_count}개")
    
    def create_combined_dataset(self):
        """통합 데이터셋 생성"""
        try:
            print("📊 통합 데이터셋 생성 중...")
            
            # 주요 컬럼만 선별
            main_columns = [
                'Open', 'High', 'Low', 'Close', 'Volume',
                'Return_1d', 'Return_7d', 'Return_30d',
                'MA_7', 'MA_20', 'MA_50',
                'EMA_12', 'EMA_26',
                'RSI_14', 'RSI_7',
                'MACD', 'MACD_Signal',
                'BB_Position_20',
                'Volatility_30d',
                'ATR_14', 'ATR_Percent_14',
                'Volume_Ratio',
                'Williams_R', 'MFI'
            ]
            
            combined_data = []
            
            for symbol, df in self.processed_data.items():
                if not df.empty:
                    # 사용 가능한 컬럼만 선택
                    available_columns = [col for col in main_columns if col in df.columns]
                    
                    if len(available_columns) >= 10:  # 최소 10개 컬럼은 있어야 함
                        temp_df = df[available_columns].copy()
                        temp_df['Symbol'] = symbol
                        temp_df['Date'] = temp_df.index
                        combined_data.append(temp_df)
            
            if combined_data:
                combined_df = pd.concat(combined_data, ignore_index=True)
                combined_df.to_csv(f"{self.output_folder}/combined_crypto_indicators.csv", index=False)
                print(f"📊 통합 데이터셋 저장 완료: {len(combined_df)}행")
            
        except Exception as e:
            print(f"⚠️ 통합 데이터셋 생성 실패: {e}")
    
    def get_indicator_summary(self):
        """계산된 지표 요약 정보"""
        if not self.processed_data:
            return "처리된 데이터가 없습니다."
        
        sample_symbol = list(self.processed_data.keys())[0]
        sample_df = self.processed_data[sample_symbol]
        
        # 지표별 그룹화
        indicator_groups = {
            '이동평균': [col for col in sample_df.columns if 'MA_' in col or 'EMA_' in col],
            'MACD': [col for col in sample_df.columns if 'MACD' in col],
            'RSI': [col for col in sample_df.columns if 'RSI' in col],
            '스토캐스틱': [col for col in sample_df.columns if 'Stoch' in col],
            'Williams %R': [col for col in sample_df.columns if 'Williams' in col],
            'CCI': [col for col in sample_df.columns if 'CCI' in col],
            'MFI': [col for col in sample_df.columns if 'MFI' in col],
            '수익률': [col for col in sample_df.columns if 'Return' in col],
            '변동성': [col for col in sample_df.columns if 'Volatility' in col or 'ATR' in col or 'BB_' in col],
            '거래량': [col for col in sample_df.columns if 'Volume' in col],
            '기타': [col for col in sample_df.columns if any(x in col for x in ['OSCP', 'DI_', 'ADX', 'Price_ROC', 'STD', 'Highest', 'Lowest'])]
        }
        
        summary = f"\n📊 계산된 기술지표 요약 ({sample_symbol} 기준)\n"
        summary += "="*50 + "\n"
        
        total_indicators = 0
        for group, indicators in indicator_groups.items():
            if indicators:
                summary += f"{group}: {len(indicators)}개\n"
                for indicator in indicators[:5]:  # 처음 5개만 표시
                    summary += f"  - {indicator}\n"
                if len(indicators) > 5:
                    summary += f"  ... 외 {len(indicators)-5}개 더\n"
                summary += "\n"
                total_indicators += len(indicators)
        
        summary += f"총 지표 수: {total_indicators}개\n"
        summary += f"처리된 코인 수: {len(self.processed_data)}개\n"
        
        return summary
    
    def run_technical_analysis(self):
        """전체 기술분석 프로세스 실행"""
        # 1. 모든 코인 처리
        self.process_all_cryptos()
        
        # 2. 결과 저장
        if self.processed_data:
            self.save_processed_data()
            
            # 3. 요약 정보 출력
            print(self.get_indicator_summary())
        else:
            print("❌ 처리된 데이터가 없습니다!")
        
        return self.processed_data

In [22]:
# 사용 예시
if __name__ == "__main__":
    # 기술지표 계산기 인스턴스 생성
    indicator_calculator = CryptoTechnicalIndicators(
        input_folder="/workspace/AI모델/projects/coin/data/1h/raw_data_1h",
        output_folder="/workspace/AI모델/projects/coin/data/1h/processed",
        start_date=start_date,  # 선택적: 계산 시작일
        end_date=end_date     # 선택적: 계산 종료일
    )
    
    # 기술분석 실행
    processed_data = indicator_calculator.run_technical_analysis()
    
    # 결과 확인
    if processed_data:
        print(f"\n🎉 기술분석 완료!")
        print(f"📊 처리된 코인: {len(processed_data)}개")
        
        # 샘플 데이터 미리보기
        sample_symbol = list(processed_data.keys())[0]
        sample_df = processed_data[sample_symbol]
        print(f"\n📋 샘플 데이터 ({sample_symbol}):")
        print(f"   - 데이터 기간: {sample_df.index[0]} ~ {sample_df.index[-1]}")
        print(f"   - 총 컬럼 수: {len(sample_df.columns)}")
        print(f"   - 최근 3일 데이터:")
        print(sample_df.tail(3)[['Close', 'MA_20', 'RSI_14', 'MACD', 'Volume_Ratio']].round(4))

🚀 기술지표 계산 시작!
📁 입력 폴더: /workspace/AI모델/projects/coin/data/1h/raw_data_1h
📅 기간: 2023-10-01 ~ 2025-09-21
📊 총 20개 파일 발견


기술지표 계산:   0%|          | 0/20 [00:00<?, ?it/s]

📊 ADA_1h 처리 중...
   - 데이터 기간: 2023-10-01 00:00:00 ~ 2025-09-20 23:00:00 (17299일)


기술지표 계산:   5%|▌         | 1/20 [00:01<00:25,  1.32s/it]

   - 최종 컬럼 수: 138
📊 AVAX_1h 처리 중...
   - 데이터 기간: 2023-10-01 00:00:00 ~ 2025-09-20 23:00:00 (17291일)


...

   - 최종 컬럼 수: 138
📊 XRP_1h 처리 중...
   - 데이터 기간: 2023-10-01 00:00:00 ~ 2025-09-20 23:00:00 (17302일)


기술지표 계산: 100%|██████████| 20/20 [00:26<00:00,  1.34s/it]


   - 최종 컬럼 수: 138
✅ 성공: 20개
❌ 실패: 0개
💾 기술지표 추가된 데이터 저장 중...


저장: 100%|██████████| 20/20 [01:10<00:00,  3.54s/it]


📊 통합 데이터셋 생성 중...
📊 통합 데이터셋 저장 완료: 345962행
✅ 기술지표 데이터 저장 완료!
📁 저장 위치: /workspace/AI모델/projects/coin/data/1h/processed/with_indicators/
✅ 저장 완료: 20개

📊 계산된 기술지표 요약 (ADA_1h 기준)
이동평균: 32개
  - MA_2
  - MA_3
  - MA_4
  - MA_5
  - MA_6
  ... 외 27개 더

MACD: 9개
  - MACD
  - MACD_Signal
  - MACD_Histogram
  - MACD_10_20
  - MACD_10_20_Signal
  ... 외 4개 더

RSI: 8개
  - RSI_4
  - RSI_5
  - RSI_6
  - RSI_7
  - RSI_12
  ... 외 3개 더

스토캐스틱: 12개
  - Stoch_K_1
  - Stoch_D_1
  - Stoch_K_3
  - Stoch_D_3
  - Stoch_K_4
  ... 외 7개 더

Williams %R: 5개
  - Williams_R_1
  - Williams_R_8
  - Williams_R_10
  - Williams_R_14
  - Williams_R

CCI: 6개
  - CCI_3
  - CCI_4
  - CCI_10
  - CCI_17
  - CCI_20
  ... 외 1개 더

MFI: 3개
  - MFI_11
  - MFI_14
  - MFI

수익률: 9개
  - Return_1d
  - Return_3d
  - Return_7d
  - Return_14d
  - Return_30d
  ... 외 4개 더

변동성: 11개
  - BB_Middle_20
  - BB_Upper_20
  - BB_Lower_20
  - BB_Width_20
  - BB_Position_20
  ... 외 6개 더

거래량: 6개
  - Volume
  - Volume_MA_7
  - Volume_MA_20
  - Volume_MA_

# 퀀트전략지표 추가

In [23]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

class QuantStrategySignals:
    def __init__(self, input_folder, output_folder):
        """
        퀀트 전략 시그널 생성기
        
        Parameters:
        input_folder: 기술지표가 포함된 CSV 파일들 폴더
        output_folder: 전략 시그널이 추가된 CSV 저장 폴더
        """
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.processed_data = {}
        self.failed_symbols = []
        
    def load_crypto_data(self, csv_file):
        """CSV 파일 로드"""
        try:
            df = pd.read_csv(csv_file)
            
            if 'Datetime' in df.columns:
                df['Datetime'] = pd.to_datetime(df['Datetime'])  # Datetime 컬럼 변환
                df.set_index('Datetime', inplace=True)
            elif 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'])
                df.set_index('Date', inplace=True)
            
            return df.sort_index()
                
        except Exception as e:
            print(f"파일 로드 실패 {csv_file}: {e}")
            return None
    
    def calculate_ma_cross_strategies(self, df):
        """이동평균 교차 전략들 - 메모 기반 수정"""
        try:
            # MA_Cross_3_25 - 메모 1-028번
            if 'MA_3' in df.columns and 'MA_25' in df.columns:
                df['MA_Cross_3_25_Signal'] = 1
                cross_up = (df['MA_3'] > df['MA_25']) & (df['MA_3'].shift(1) <= df['MA_25'].shift(1))
                cross_down = (df['MA_3'] < df['MA_25']) & (df['MA_3'].shift(1) >= df['MA_25'].shift(1))
                df.loc[cross_up, 'MA_Cross_3_25_Signal'] = 2
                df.loc[cross_down, 'MA_Cross_3_25_Signal'] = 0
            
            # MA_Trend_20_60_120 - 메모 1-019번
            if all(col in df.columns for col in ['MA_20', 'MA_60', 'MA_120']):
                uptrend = (df['MA_20'] > df['MA_60']) & (df['MA_60'] > df['MA_120'])
                downtrend = (df['MA_20'] < df['MA_60']) & (df['MA_60'] < df['MA_120'])
                df['MA_Trend_Signal'] = 1
                df.loc[uptrend, 'MA_Trend_Signal'] = 2
                df.loc[downtrend, 'MA_Trend_Signal'] = 0
            
            # EMA_Cross_5_20 - 메모 2-114번
            if 'EMA_5' in df.columns and 'EMA_20' in df.columns:
                df['EMA_Cross_5_20_Signal'] = 1
                cross_up = (df['EMA_5'] > df['EMA_20']) & (df['EMA_5'].shift(1) <= df['EMA_20'].shift(1))
                cross_down = (df['EMA_5'] < df['EMA_20']) & (df['EMA_5'].shift(1) >= df['EMA_20'].shift(1))
                df.loc[cross_up, 'EMA_Cross_5_20_Signal'] = 2
                df.loc[cross_down, 'EMA_Cross_5_20_Signal'] = 0
            
            # EMA_Cross_6_24 - 메모 3-253번 골든크로스
            if 'EMA_6' in df.columns and 'EMA_24' in df.columns:
                df['EMA_Cross_6_24_Signal'] = 1
                cross_up = (df['EMA_6'] > df['EMA_24']) & (df['EMA_6'].shift(1) <= df['EMA_24'].shift(1))
                cross_down = (df['EMA_6'] < df['EMA_24']) & (df['EMA_6'].shift(1) >= df['EMA_24'].shift(1))
                df.loc[cross_up, 'EMA_Cross_6_24_Signal'] = 2
                df.loc[cross_down, 'EMA_Cross_6_24_Signal'] = 0
            
            # 정진 전략 - 메모 1-135번
            if 'MA_2' in df.columns and 'MA_29' in df.columns:
                df['Jungjin_Signal'] = 1
                # 당일 종가가 2일전의 2일/29일 이평선보다 클 때
                ma2_2days_ago = df['MA_2'].shift(2)
                ma29_2days_ago = df['MA_29'].shift(2)
                buy_condition = (df['Close'] > ma2_2days_ago) & (df['Close'] > ma29_2days_ago)
                sell_condition = (df['Close'] < ma2_2days_ago) & (df['Close'] < ma29_2days_ago)
                df.loc[buy_condition, 'Jungjin_Signal'] = 2
                df.loc[sell_condition, 'Jungjin_Signal'] = 0
                
        except Exception as e:
            print(f"이동평균 전략 계산 실패: {e}")

    def calculate_macd_strategies(self, df):
        """MACD 기반 전략들 - 다양한 매개변수 버전 추가"""
        try:
            # 기본 MACD_Zero_Cross (12,26)
            if 'MACD' in df.columns:
                df['MACD_Zero_Cross_Signal'] = 1
                zero_cross_up = (df['MACD'] > 0) & (df['MACD'].shift(1) <= 0)
                zero_cross_down = (df['MACD'] < 0) & (df['MACD'].shift(1) >= 0)
                df.loc[zero_cross_up, 'MACD_Zero_Cross_Signal'] = 2
                df.loc[zero_cross_down, 'MACD_Zero_Cross_Signal'] = 0
            
            # MACD (10,20) 변형 - 메모 1-58번 전략
            if 'MACD_10_20' in df.columns:
                df['MACD_10_20_Signal'] = 1
                zero_cross_up_10_20 = (df['MACD_10_20'] > 0) & (df['MACD_10_20'].shift(1) <= 0)
                zero_cross_down_10_20 = (df['MACD_10_20'] < 0) & (df['MACD_10_20'].shift(1) >= 0)
                df.loc[zero_cross_up_10_20, 'MACD_10_20_Signal'] = 2
                df.loc[zero_cross_down_10_20, 'MACD_10_20_Signal'] = 0
            
            # MACD (15,26) 변형 - 메모 1-144번 전략
            if 'MACD_15_26' in df.columns:
                df['MACD_15_26_Signal'] = 1
                zero_cross_up_15_26 = (df['MACD_15_26'] > 0) & (df['MACD_15_26'].shift(1) <= 0)
                zero_cross_down_15_26 = (df['MACD_15_26'] < 0) & (df['MACD_15_26'].shift(1) >= 0)
                df.loc[zero_cross_up_15_26, 'MACD_15_26_Signal'] = 2
                df.loc[zero_cross_down_15_26, 'MACD_15_26_Signal'] = 0
            
            # MACD (5,27) 변형 - 메모 3-292번 전략
            if 'MACD_5_27' in df.columns:
                df['MACD_5_27_Signal'] = 1
                zero_cross_up_5_27 = (df['MACD_5_27'] > 0) & (df['MACD_5_27'].shift(1) <= 0)
                zero_cross_down_5_27 = (df['MACD_5_27'] < 0) & (df['MACD_5_27'].shift(1) >= 0)
                df.loc[zero_cross_up_5_27, 'MACD_5_27_Signal'] = 2
                df.loc[zero_cross_down_5_27, 'MACD_5_27_Signal'] = 0
            
            # MACD 시그널 교차
            if 'MACD' in df.columns and 'MACD_Signal' in df.columns:
                df['MACD_Signal_Cross'] = 1
                signal_cross_up = (df['MACD'] > df['MACD_Signal']) & (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1))
                signal_cross_down = (df['MACD'] < df['MACD_Signal']) & (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1))
                df.loc[signal_cross_up, 'MACD_Signal_Cross'] = 2
                df.loc[signal_cross_down, 'MACD_Signal_Cross'] = 0
            
            # Bad Market3 전략 (2-114): EMA + MACD 조합
            if all(col in df.columns for col in ['EMA_5', 'EMA_20', 'MACD', 'MACD_Signal']):
                df['Bad_Market3_Signal'] = 1
                
                # EMA 상향돌파 & MACD 시그널 상향돌파
                ema_cross_up = (df['EMA_5'] > df['EMA_20']) & (df['EMA_5'].shift(1) <= df['EMA_20'].shift(1))
                macd_signal_up = (df['MACD'] > df['MACD_Signal']) & (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1))
                buy_condition = ema_cross_up & macd_signal_up
                
                # EMA 하향돌파 & MACD 시그널 하향돌파  
                if 'EMA_10' in df.columns:
                    ema_cross_down = (df['EMA_5'] < df['EMA_10']) & (df['EMA_5'].shift(1) >= df['EMA_10'].shift(1))
                    macd_signal_down = (df['MACD'] < df['MACD_Signal']) & (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1))
                    sell_condition = ema_cross_down & macd_signal_down
                    df.loc[sell_condition, 'Bad_Market3_Signal'] = 0
                
                df.loc[buy_condition, 'Bad_Market3_Signal'] = 2
                
        except Exception as e:
            print(f"MACD 전략 계산 실패: {e}")

    def calculate_rsi_strategies(self, df):
        """RSI 기반 전략들 - 다양한 매개변수와 임계값"""
        try:
            # 기본 RSI_Reversal (30/70)
            if 'RSI_14' in df.columns:
                df['RSI_Reversal_Signal'] = 1
                oversold_bounce = (df['RSI_14'] > 30) & (df['RSI_14'].shift(1) <= 30)
                overbought_drop = (df['RSI_14'] < 70) & (df['RSI_14'].shift(1) >= 70)
                df.loc[oversold_bounce, 'RSI_Reversal_Signal'] = 2
                df.loc[overbought_drop, 'RSI_Reversal_Signal'] = 0
            
            # RSI_Extreme (20/80)
            if 'RSI_14' in df.columns:
                df['RSI_Extreme_Signal'] = 1
                extreme_oversold = (df['RSI_14'] > 20) & (df['RSI_14'].shift(1) <= 20)
                extreme_overbought = (df['RSI_14'] < 80) & (df['RSI_14'].shift(1) >= 80)
                df.loc[extreme_oversold, 'RSI_Extreme_Signal'] = 2
                df.loc[extreme_overbought, 'RSI_Extreme_Signal'] = 0
            
            # 리버스 RSI (25/64) - 메모 1-178번
            if 'RSI_14' in df.columns:
                df['RSI_Reverse_Signal'] = 1
                reverse_buy = (df['RSI_14'] > 25) & (df['RSI_14'].shift(1) <= 25)
                reverse_sell = (df['RSI_14'] < 64) & (df['RSI_14'].shift(1) >= 64)
                df.loc[reverse_buy, 'RSI_Reverse_Signal'] = 2
                df.loc[reverse_sell, 'RSI_Reverse_Signal'] = 0
            
            # RSI (20/75) - 메모 3-326번
            if 'RSI_21' in df.columns:
                df['RSI_20_75_Signal'] = 1
                rsi_20_buy = (df['RSI_21'] > 20) & (df['RSI_21'].shift(1) <= 20)
                rsi_75_sell = (df['RSI_21'] < 75) & (df['RSI_21'].shift(1) >= 75)
                df.loc[rsi_20_buy, 'RSI_20_75_Signal'] = 2
                df.loc[rsi_75_sell, 'RSI_20_75_Signal'] = 0
            
            # RSI (22/78) - 메모 3-143번
            if 'RSI_20' in df.columns:
                df['RSI_22_78_Signal'] = 1
                rsi_22_buy = (df['RSI_20'] > 22) & (df['RSI_20'].shift(1) <= 22)
                rsi_78_sell = (df['RSI_20'] < 78) & (df['RSI_20'].shift(1) >= 78)
                df.loc[rsi_22_buy, 'RSI_22_78_Signal'] = 2
                df.loc[rsi_78_sell, 'RSI_22_78_Signal'] = 0
            
            # RSI (30/65) - 메모 4-304번
            if 'RSI_4' in df.columns:
                df['RSI_30_65_Signal'] = 1
                rsi_30_buy = (df['RSI_4'] > 30) & (df['RSI_4'].shift(1) <= 30)
                rsi_65_sell = (df['RSI_4'] < 65) & (df['RSI_4'].shift(1) >= 65)
                df.loc[rsi_30_buy, 'RSI_30_65_Signal'] = 2
                df.loc[rsi_65_sell, 'RSI_30_65_Signal'] = 0
            
            # RSI (12/50) - 메모 9-583번
            if 'RSI_12' in df.columns:
                df['RSI_12_50_Signal'] = 1
                rsi_12_buy = (df['RSI_12'] > 12) & (df['RSI_12'].shift(1) <= 12)
                rsi_50_sell = (df['RSI_12'] < 50) & (df['RSI_12'].shift(1) >= 50)
                df.loc[rsi_12_buy, 'RSI_12_50_Signal'] = 2
                df.loc[rsi_50_sell, 'RSI_12_50_Signal'] = 0
                
        except Exception as e:
            print(f"RSI 전략 계산 실패: {e}")

    def calculate_oscillator_strategies(self, df):
        """오실레이터 복합 전략들 - 메모 기반 수정"""
        try:
            # Williams_CCI_Combo - 메모 3-191번 기반
            if 'Williams_R_10' in df.columns and 'CCI_10' in df.columns:
                df['Williams_CCI_Signal'] = 1
                
                # W%R -96 돌파 & CCI -137 돌파
                williams_buy = (df['Williams_R_10'] > -96) & (df['Williams_R_10'].shift(1) <= -96)
                cci_buy = (df['CCI_10'] > -137) & (df['CCI_10'].shift(1) <= -137)
                combo_buy = williams_buy & cci_buy
                
                # W%R -32 하향돌파 & CCI 63 하향돌파
                williams_sell = (df['Williams_R_10'] < -32) & (df['Williams_R_10'].shift(1) >= -32)
                cci_sell = (df['CCI_10'] < 63) & (df['CCI_10'].shift(1) >= 63)
                combo_sell = williams_sell & cci_sell
                
                df.loc[combo_buy, 'Williams_CCI_Signal'] = 2
                df.loc[combo_sell, 'Williams_CCI_Signal'] = 0
            
            # CCI 과매도/과매수 - 메모 3-026번
            if 'CCI_17' in df.columns: # CCI_3 조건은 제거해도 무방
                df['CCI_Oversold_Signal'] = 1
                # CCI_17을 기준으로 통일
                cci_buy = (df['CCI_17'] > -100) & (df['CCI_17'].shift(1) <= -100)
                cci_sell = (df['CCI_17'] < 100) & (df['CCI_17'].shift(1) >= 100)
                df.loc[cci_buy, 'CCI_Oversold_Signal'] = 2
                df.loc[cci_sell, 'CCI_Oversold_Signal'] = 0
            
            # CCI (3) -160/120 - 메모 4-362번
            if 'CCI_3' in df.columns:
                df['CCI_3_Signal'] = 1
                cci_3_buy = (df['CCI_3'] > -160) & (df['CCI_3'].shift(1) <= -160)
                cci_3_sell = (df['CCI_3'] < 120) & (df['CCI_3'].shift(1) >= 120)
                df.loc[cci_3_buy, 'CCI_3_Signal'] = 2
                df.loc[cci_3_sell, 'CCI_3_Signal'] = 0
            
            # Stoch_RSI_Combo - 메모 2-249번 기반
            if all(col in df.columns for col in ['Stoch_K_4', 'Stoch_D_4', 'RSI_5']):
                df['Stoch_RSI_Combo_Signal'] = 1
                
                # %K가 %D 상향돌파 & RSI 15 상향돌파
                stoch_cross_up = (df['Stoch_K_4'] > df['Stoch_D_4']) & (df['Stoch_K_4'].shift(1) <= df['Stoch_D_4'].shift(1))
                rsi_bounce = (df['RSI_5'] > 15) & (df['RSI_5'].shift(1) <= 15)
                buy_condition = stoch_cross_up & rsi_bounce
                
                # %K가 %D 하향돌파 & RSI 65 하향돌파
                stoch_cross_down = (df['Stoch_K_4'] < df['Stoch_D_4']) & (df['Stoch_K_4'].shift(1) >= df['Stoch_D_4'].shift(1))
                rsi_drop = (df['RSI_5'] < 65) & (df['RSI_5'].shift(1) >= 65)
                sell_condition = stoch_cross_down & rsi_drop
                
                df.loc[buy_condition, 'Stoch_RSI_Combo_Signal'] = 2
                df.loc[sell_condition, 'Stoch_RSI_Combo_Signal'] = 0
            
            # 스토캐스틱 단독 전략들
            # 스토캐스틱 10/72 - 메모 3-231번
            if 'Stoch_K_3' in df.columns:
                df['Stoch_10_72_Signal'] = 1
                stoch_10_buy = (df['Stoch_K_3'] > 10) & (df['Stoch_K_3'].shift(1) <= 10)
                stoch_72_sell = (df['Stoch_K_3'] < 72) & (df['Stoch_K_3'].shift(1) >= 72)
                df.loc[stoch_10_buy, 'Stoch_10_72_Signal'] = 2
                df.loc[stoch_72_sell, 'Stoch_10_72_Signal'] = 0
            
            # 스토캐스틱 71/31 - 메모 8-432번
            if 'Stoch_K_1' in df.columns:
                df['Stoch_71_31_Signal'] = 1
                stoch_71_buy = (df['Stoch_K_1'] > 71) & (df['Stoch_K_1'].shift(1) <= 71)
                stoch_31_sell = (df['Stoch_K_1'] < 31) & (df['Stoch_K_1'].shift(1) >= 31)
                df.loc[stoch_71_buy, 'Stoch_71_31_Signal'] = 0
                df.loc[stoch_31_sell, 'Stoch_71_31_Signal'] = 2
                
        except Exception as e:
            print(f"오실레이터 전략 계산 실패: {e}")

    def calculate_price_pattern_strategies(self, df):
        """가격 패턴 기반 전략들"""
        try:
            if all(col in df.columns for col in ['Open', 'High', 'Low', 'Close']):
                # 기본 캔들스틱 패턴
                bullish_candle = (df['Close'] > df['Open']) & \
                               (abs(df['Close'] - df['Open']) > 0.01) & \
                               (abs(df['Close'] - df['Open']) < 0.05)
                bearish_candle = (df['Close'] < df['Open']) & \
                               (abs(df['Close'] - df['Open']) < 0.05)
                
                df['Candlestick_Signal'] = 1
                df.loc[bullish_candle, 'Candlestick_Signal'] = 2
                df.loc[bearish_candle, 'Candlestick_Signal'] = 0
                
                # 수식3 전략 - 메모 7-334번
                df['Formula3_Signal'] = 1
                formula3_buy = (df['Close'] > df['Open']) & \
                              (abs(df['Close'] - df['Open']) > 0.03) & \
                              (abs(df['Close'] - df['Open']) < 0.5)
                formula3_sell = (df['Close'] < df['Open']) & \
                               (abs(df['Close'] - df['Open']) < 0.5)
                
                df.loc[formula3_buy, 'Formula3_Signal'] = 2
                df.loc[formula3_sell, 'Formula3_Signal'] = 0
                
                # Shadow Analysis
                upper_shadow = np.where(df['Close'] > df['Open'], 
                                      df['High'] - df['Close'], 
                                      df['High'] - df['Open'])
                lower_shadow = np.where(df['Close'] > df['Open'], 
                                      df['Open'] - df['Low'], 
                                      df['Close'] - df['Low'])
                
                long_lower_shadow = lower_shadow > upper_shadow * 2
                long_upper_shadow = upper_shadow > lower_shadow * 2
                
                df['Shadow_Analysis_Signal'] = 1
                df.loc[long_lower_shadow, 'Shadow_Analysis_Signal'] = 2
                df.loc[long_upper_shadow, 'Shadow_Analysis_Signal'] = 0
                
        except Exception as e:
            print(f"가격 패턴 전략 계산 실패: {e}")
    
    def calculate_pivot_strategy(self, df):
        """피봇 지지/저항 전략"""
        try:
            if all(col in df.columns for col in ['High', 'Low', 'Close']):
                # 피봇 포인트 계산
                df['Pivot'] = (df['High'] + df['Low'] + df['Close']) / 3
                df['Support1'] = 2 * df['Pivot'] - df['High']
                df['Resistance1'] = 2 * df['Pivot'] - df['Low']
                
                # 지지선 돌파 (상향)
                support_breakout = (df['Close'] > df['Support1']) & (df['Open'] <= df['Support1'])
                # 저항선 이탈 (하향)
                resistance_breakdown = (df['Close'] < df['Resistance1']) & (df['Open'] >= df['Resistance1'])
                
                df['Pivot_Strategy_Signal'] = 1
                df.loc[support_breakout, 'Pivot_Strategy_Signal'] = 2
                df.loc[resistance_breakdown, 'Pivot_Strategy_Signal'] = 0
                
        except Exception as e:
            print(f"피봇 전략 계산 실패: {e}")
    
    def calculate_volume_strategies(self, df):
        """거래량 기반 전략들 - MFI 전략 세분화"""
        try:
            # 기본 MFI_Strategy (20/80)
            if 'MFI' in df.columns:
                df['MFI_Strategy_Signal'] = 1
                mfi_oversold = (df['MFI'] > 20) & (df['MFI'].shift(1) <= 20)
                mfi_overbought = (df['MFI'] < 80) & (df['MFI'].shift(1) >= 80)
                df.loc[mfi_oversold, 'MFI_Strategy_Signal'] = 2
                df.loc[mfi_overbought, 'MFI_Strategy_Signal'] = 0
            
            # MFI (25/50) - 메모 3-131번
            if 'MFI_11' in df.columns:
                df['MFI_25_50_Signal'] = 1
                mfi_25_buy = (df['MFI_11'] > 25) & (df['MFI_11'].shift(1) <= 25)
                mfi_50_sell = (df['MFI_11'] < 50) & (df['MFI_11'].shift(1) >= 50)
                df.loc[mfi_25_buy, 'MFI_25_50_Signal'] = 2
                df.loc[mfi_50_sell, 'MFI_25_50_Signal'] = 0
            
            # Money Flow Index 극값 - 메모 4-363번
            if 'MFI_11' in df.columns:
                df['MFI_Extreme_Signal'] = 1
                df.loc[df['MFI_11'] <= 15, 'MFI_Extreme_Signal'] = 2
                df.loc[df['MFI_11'] >= 85, 'MFI_Extreme_Signal'] = 0
            
            # Volume_Breakout
            if 'Volume_Ratio' in df.columns:
                df['Volume_Breakout_Signal'] = 1
                volume_surge = (df['Volume_Ratio'] > 2.0) & (df['Volume_Ratio'].shift(1) <= 2.0)
                volume_dry = (df['Volume_Ratio'] < 0.5) & (df['Volume_Ratio'].shift(1) >= 0.5)
                df.loc[volume_surge, 'Volume_Breakout_Signal'] = 2
                df.loc[volume_dry, 'Volume_Breakout_Signal'] = 0
                
        except Exception as e:
            print(f"거래량 전략 계산 실패: {e}")

    def calculate_momentum_strategies(self, df):
        """모멘텀 전략들 - Price ROC 추가"""
        try:
            # 기존 Price_ROC (Return_7d 기반)
            if 'Return_7d' in df.columns:
                df['Momentum_Signal'] = 1
                strong_momentum_up = (df['Return_7d'] > 0.1) & (df['Return_7d'].shift(1) <= 0.1)
                strong_momentum_down = (df['Return_7d'] < -0.1) & (df['Return_7d'].shift(1) >= -0.1)
                df.loc[strong_momentum_up, 'Momentum_Signal'] = 2
                df.loc[strong_momentum_down, 'Momentum_Signal'] = 0
            
            # Price ROC (3) - 메모 10-540번
            if 'Price_ROC_3' in df.columns:
                df['Price_ROC_3_Signal'] = 1
                roc_up = (df['Price_ROC_3'] > 0) & (df['Price_ROC_3'].shift(1) <= 0)
                roc_down = (df['Price_ROC_3'] < 0) & (df['Price_ROC_3'].shift(1) >= 0)
                df.loc[roc_up, 'Price_ROC_3_Signal'] = 2
                df.loc[roc_down, 'Price_ROC_3_Signal'] = 0
            
            # Volatility_Breakout
            if 'ATR_Percent_14' in df.columns:
                high_volatility = df['ATR_Percent_14'] > df['ATR_Percent_14'].rolling(20).quantile(0.8)
                low_volatility = df['ATR_Percent_14'] < df['ATR_Percent_14'].rolling(20).quantile(0.2)
                df['Volatility_Signal'] = 1
                df.loc[high_volatility, 'Volatility_Signal'] = 2
                df.loc[low_volatility, 'Volatility_Signal'] = 0
                
        except Exception as e:
            print(f"모멘텀 전략 계산 실패: {e}")
    
    def calculate_composite_signals(self, df):
        """복합 시그널 계산"""
        try:
            # 모든 전략 시그널 컬럼 찾기
            signal_columns = [col for col in df.columns if '_Signal' in col and col.endswith('_Signal')]
            
            if len(signal_columns) > 0:
                # 평균 시그널 강도
                df['Composite_Signal_Avg'] = df[signal_columns].mean(axis=1)
                
                # 매수 시그널 개수
                buy_signals = (df[signal_columns] == 2).sum(axis=1)
                sell_signals = (df[signal_columns] == 0).sum(axis=1)
                
                df['Buy_Signal_Count'] = buy_signals
                df['Sell_Signal_Count'] = sell_signals
                df['Net_Signal_Score'] = buy_signals - sell_signals
                
                # 최종 복합 시그널
                df['Final_Composite_Signal'] = 1  # 기본 HOLD
                df.loc[df['Net_Signal_Score'] >= 2, 'Final_Composite_Signal'] = 2  # 강한 매수
                df.loc[df['Net_Signal_Score'] <= -2, 'Final_Composite_Signal'] = 0  # 강한 매도
                
        except Exception as e:
            print(f"복합 시그널 계산 실패: {e}")
    
    def process_single_crypto(self, csv_file):
        """개별 코인 처리"""
        symbol = os.path.basename(csv_file).replace('.csv', '')
        
        try:
            print(f"전략 시그널 생성: {symbol}")
            
            df = self.load_crypto_data(csv_file)
            if df is None or len(df) < 50:
                self.failed_symbols.append(symbol)
                return None
            
            print(f"   - 데이터 기간: {df.index[0]} ~ {df.index[-1]} ({len(df)}일)")
            
            # 각 전략 계산
            self.calculate_ma_cross_strategies(df)
            self.calculate_macd_strategies(df)
            self.calculate_rsi_strategies(df)
            self.calculate_oscillator_strategies(df)
            self.calculate_price_pattern_strategies(df)
            self.calculate_pivot_strategy(df)
            self.calculate_volume_strategies(df)
            self.calculate_momentum_strategies(df)
            
            # 복합 시그널 계산
            self.calculate_composite_signals(df)
            
            # 결과 저장
            self.processed_data[symbol] = df
            
            # 새로 추가된 시그널 컬럼 수 계산
            signal_columns = [col for col in df.columns if '_Signal' in col]
            print(f"   - 생성된 전략 시그널: {len(signal_columns)}개")
            
            return df
            
        except Exception as e:
            print(f"⚠️ {symbol} 처리 실패: {e}")
            self.failed_symbols.append(symbol)
            return None
    
    def process_all_cryptos(self):
        """모든 코인 처리"""
        print("🚀 퀀트 전략 시그널 생성 시작!")
        print(f"📁 입력 폴더: {self.input_folder}")
        print("="*50)
        
        csv_files = [f for f in os.listdir(self.input_folder) if f.endswith('.csv')]
        
        if not csv_files:
            print("❌ CSV 파일을 찾을 수 없습니다!")
            return
        
        print(f"📊 총 {len(csv_files)}개 파일 발견")
        
        for csv_file in tqdm(csv_files, desc="전략 시그널 생성"):
            file_path = os.path.join(self.input_folder, csv_file)
            self.process_single_crypto(file_path)
        
        print("="*50)
        print(f"✅ 성공: {len(self.processed_data)}개")
        print(f"❌ 실패: {len(self.failed_symbols)}개")
    
    def save_strategy_data(self):
        """전략 시그널 데이터 저장"""
        print("💾 전략 시그널 데이터 저장 중...")
        
        os.makedirs(self.output_folder, exist_ok=True)
        os.makedirs(f"{self.output_folder}/with_strategies", exist_ok=True)
        
        saved_count = 0
        
        for symbol, df in tqdm(self.processed_data.items(), desc="저장"):
            try:
                df_to_save = df.copy()
                df_to_save.reset_index(inplace=True)
                
                filename = f"{self.output_folder}/with_strategies/{symbol}.csv"
                df_to_save.to_csv(filename, index=False)
                saved_count += 1
                
            except Exception as e:
                print(f"⚠️ {symbol} 저장 실패: {e}")
        
        # 통합 데이터셋 생성
        if self.processed_data:
            self.create_ml_dataset()
        
        # 요약 저장
        summary = {
            'total_processed': len(self.processed_data),
            'total_failed': len(self.failed_symbols),
            'failed_symbols': self.failed_symbols,
            'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        summary_df = pd.DataFrame([summary])
        summary_df.to_csv(f"{self.output_folder}/strategy_summary.csv", index=False)
        
        print(f"✅ 전략 시그널 데이터 저장 완료!")
        print(f"📁 저장 위치: {self.output_folder}/with_strategies/")
        print(f"✅ 저장 완료: {saved_count}개")
    
    def create_ml_dataset(self):
        """머신러닝용 통합 데이터셋 생성 - 수정 버전"""
        try:
            print("🤖 머신러닝용 데이터셋 생성 중...")
            
            # 머신러닝용 주요 컬럼 선별
            ml_columns = [
                # 기본 가격 데이터
                'Open', 'High', 'Low', 'Close', 'Volume',
                
                # 기술지표
                'MA_7', 'MA_20', 'MA_50', 'EMA_12', 'EMA_26',
                'RSI_14', 'RSI_7', 'MACD', 'MACD_Signal',
                'BB_Position_20', 'Volatility_30d', 'ATR_Percent_14',
                'Volume_Ratio', 'Williams_R', 'MFI',
                
                # 수익률
                'Return_1d', 'Return_7d', 'Return_30d',
                
                # 전략 시그널들 (실제 생성된 컬럼명에 맞춰 수정)
                'MA_Cross_3_25_Signal', 'MA_Trend_Signal', 'EMA_Cross_5_20_Signal',
                'MACD_Zero_Cross_Signal', 'MACD_Signal_Cross',
                'RSI_Reversal_Signal', 'RSI_Extreme_Signal',
                'Williams_CCI_Signal', 'Stoch_RSI_Combo_Signal',
                'Candlestick_Signal', 'Shadow_Analysis_Signal',
                'Pivot_Strategy_Signal', 'MFI_Strategy_Signal',
                'Volume_Breakout_Signal', 'Momentum_Signal',
                'Volatility_Signal', 'Final_Composite_Signal',
                
                # 메타 정보
                'Buy_Signal_Count', 'Sell_Signal_Count', 'Net_Signal_Score'
            ]
            
            combined_data = []
            
            for symbol, df in self.processed_data.items():
                if not df.empty:
                    # 사용 가능한 컬럼만 선택
                    available_columns = [col for col in ml_columns if col in df.columns]
                    
                    if len(available_columns) >= 20:  # 최소 20개 특성은 있어야 함
                        temp_df = df[available_columns].copy()
                        temp_df['Symbol'] = symbol
                        temp_df['Date'] = temp_df.index
                        
                        # NaN 값 처리
                        temp_df = temp_df.dropna()
                        
                        if len(temp_df) > 100:  # 충분한 데이터가 있는 경우만
                            combined_data.append(temp_df)
            
            if combined_data:
                combined_df = pd.concat(combined_data, ignore_index=True)
                
                # ✅ 올바른 미래 수익률 계산
                combined_df = combined_df.sort_values(['Symbol', 'Date'])
                
                # 방법 1: 7일 후 종가를 먼저 계산
                combined_df['Future_Close_7d'] = combined_df.groupby('Symbol')['Close'].shift(-7)
                
                # 미래 수익률 계산: (미래가격 / 현재가격) - 1
                combined_df['Future_Return_7d'] = (combined_df['Future_Close_7d'] / combined_df['Close']) - 1
                
                # 타겟 라벨 생성 (0: SELL, 1: HOLD, 2: BUY)
                combined_df['Target_Label'] = 1  # 기본값: 보합 (HOLD)
                combined_df.loc[combined_df['Future_Return_7d'] > 0.05, 'Target_Label'] = 2   # 5% 이상 상승 -> BUY
                combined_df.loc[combined_df['Future_Return_7d'] < -0.05, 'Target_Label'] = 0  # 5% 이상 하락 -> SELL
                
                # NaN 제거 (마지막 7일치는 미래값이 없으므로 제거)
                combined_df = combined_df.dropna(subset=['Future_Return_7d', 'Target_Label'])
                
                # Future_Close_7d 컬럼은 제거 (학습에 사용하면 안 됨 - 미래 정보 누수)
                combined_df = combined_df.drop(columns=['Future_Close_7d'])
                
                # 저장
                combined_df.to_csv(f"{self.output_folder}/ml_dataset_with_strategies.csv", index=False)
                print(f"🤖 머신러닝 데이터셋 저장 완료: {len(combined_df)}행, {len(combined_df.columns)}개 특성")
                
                # 타겟 분포 출력
                target_dist = combined_df['Target_Label'].value_counts().sort_index()
                print(f"   📊 타겟 분포 - SELL(0): {target_dist.get(0, 0)}, HOLD(1): {target_dist.get(1, 0)}, BUY(2): {target_dist.get(2, 0)}")
                
                # 추가: 타겟 라벨 검증 정보 출력
                if len(combined_df) > 0:
                    sample_idx = len(combined_df) // 2  # 중간 샘플 확인
                    print(f"   🔍 검증 샘플:")
                    print(f"      - 현재가: {combined_df.iloc[sample_idx]['Close']:.2f}")
                    print(f"      - 미래 수익률: {combined_df.iloc[sample_idx]['Future_Return_7d']*100:.2f}%")
                    print(f"      - 타겟 라벨: {combined_df.iloc[sample_idx]['Target_Label']} ({['SELL', 'HOLD', 'BUY'][int(combined_df.iloc[sample_idx]['Target_Label'])]})")
            
        except Exception as e:
            print(f"⚠️ ML 데이터셋 생성 실패: {e}")
        
    def get_strategy_summary(self):
        """전략 요약 정보"""
        if not self.processed_data:
            return "처리된 데이터가 없습니다."
        
        sample_symbol = list(self.processed_data.keys())[0]
        sample_df = self.processed_data[sample_symbol]
        
        strategy_columns = [col for col in sample_df.columns if '_Signal' in col]
        
        summary = f"\n📊 생성된 퀀트 전략 시그널 요약 ({sample_symbol} 기준)\n"
        summary += "="*50 + "\n"
        
        strategy_groups = {
            '이동평균 전략': [col for col in strategy_columns if 'MA_' in col or 'EMA_' in col],
            'MACD 전략': [col for col in strategy_columns if 'MACD' in col],
            'RSI 전략': [col for col in strategy_columns if 'RSI' in col],
            '오실레이터 전략': [col for col in strategy_columns if any(x in col for x in ['Williams', 'CCI', 'Stoch'])],
            '가격패턴 전략': [col for col in strategy_columns if any(x in col for x in ['Candlestick', 'Shadow', 'Pivot', 'Formula'])],
            '거래량 전략': [col for col in strategy_columns if any(x in col for x in ['MFI', 'Volume'])],
            '모멘텀 전략': [col for col in strategy_columns if any(x in col for x in ['Momentum', 'Volatility', 'ROC'])],
            '복합 전략': [col for col in strategy_columns if 'Composite' in col or 'Final' in col]
        }
        
        for group, strategies in strategy_groups.items():
            if strategies:
                summary += f"{group}: {len(strategies)}개\n"
                for strategy in strategies:
                    # 각 시그널의 분포 계산
                    if strategy in sample_df.columns:
                        signal_dist = sample_df[strategy].value_counts().sort_index()
                        buy_pct = (signal_dist.get(2, 0) / len(sample_df) * 100) if len(sample_df) > 0 else 0
                        sell_pct = (signal_dist.get(0, 0) / len(sample_df) * 100) if len(sample_df) > 0 else 0
                        hold_pct = (signal_dist.get(1, 0) / len(sample_df) * 100) if len(sample_df) > 0 else 0
                        summary += f"  - {strategy}: BUY {buy_pct:.1f}%, HOLD {hold_pct:.1f}%, SELL {sell_pct:.1f}%\n"
                summary += "\n"
        
        summary += f"총 전략 시그널: {len(strategy_columns)}개\n"
        summary += f"처리된 코인 수: {len(self.processed_data)}개\n"
        
        return summary
    
    def run_strategy_analysis(self):
        """전체 전략 분석 프로세스 실행"""
        # 1. 모든 코인 처리
        self.process_all_cryptos()
        
        # 2. 결과 저장
        if self.processed_data:
            self.save_strategy_data()
            
            # 3. 요약 정보 출력
            print(self.get_strategy_summary())
        else:
            print("❌ 처리된 데이터가 없습니다!")
        
        return self.processed_data

In [24]:


# 사용 예시
if __name__ == "__main__":
    # 퀀트 전략 시그널 생성기 인스턴스 생성
    strategy_generator = QuantStrategySignals(
        input_folder="/workspace/AI모델/projects/coin/data/1h/processed/with_indicators",
        output_folder="/workspace/AI모델/projects/coin/data/1h/final"
    )
    
    # 전략 분석 실행
    processed_data = strategy_generator.run_strategy_analysis()
    
    # 결과 확인
    if processed_data:
        print(f"\n🎉 퀀트 전략 시그널 생성 완료!")
        print(f"📊 처리된 코인: {len(processed_data)}개")
        
        # 샘플 데이터 미리보기
        sample_symbol = list(processed_data.keys())[0]
        sample_df = processed_data[sample_symbol]
        
        # 전략 시그널 컬럼들
        signal_columns = [col for col in sample_df.columns if '_Signal' in col]
        
        print(f"\n📋 샘플 데이터 ({sample_symbol}):")
        print(f"   - 데이터 기간: {sample_df.index[0]} ~ {sample_df.index[-1]}")
        print(f"   - 총 컬럼 수: {len(sample_df.columns)}")
        print(f"   - 전략 시그널 수: {len(signal_columns)}")
        
        print(f"\n📊 최근 5일 주요 시그널:")
        display_columns = ['Close', 'Final_Composite_Signal', 'Buy_Signal_Count', 'Sell_Signal_Count', 'Net_Signal_Score']
        available_display = [col for col in display_columns if col in sample_df.columns]
        
        if available_display:
            print(sample_df.tail(5)[available_display].round(4))
        
        print(f"\n💡 머신러닝 데이터셋도 생성되었습니다!")
        print(f"   파일: /final/ml_dataset_with_strategies.csv")

🚀 퀀트 전략 시그널 생성 시작!
📁 입력 폴더: /workspace/AI모델/projects/coin/data/1h/processed/with_indicators
📊 총 20개 파일 발견


전략 시그널 생성:   0%|          | 0/20 [00:00<?, ?it/s]

전략 시그널 생성: ADA_1h


전략 시그널 생성:   5%|▌         | 1/20 [00:00<00:12,  1.49it/s]

   - 데이터 기간: 2023-10-01 00:00:00 ~ 2025-09-20 23:00:00 (17299일)
   - 생성된 전략 시그널: 41개
전략 시그널 생성: AVAX_1h


...

   - 데이터 기간: 2023-10-01 00:00:00 ~ 2025-09-20 23:00:00 (17299일)
   - 생성된 전략 시그널: 41개
전략 시그널 생성: XRP_1h


전략 시그널 생성: 100%|██████████| 20/20 [00:14<00:00,  1.42it/s]


   - 데이터 기간: 2023-10-01 00:00:00 ~ 2025-09-20 23:00:00 (17302일)
   - 생성된 전략 시그널: 41개
✅ 성공: 20개
❌ 실패: 0개
💾 전략 시그널 데이터 저장 중...


저장: 100%|██████████| 20/20 [01:11<00:00,  3.59s/it]


🤖 머신러닝용 데이터셋 생성 중...
🤖 머신러닝 데이터셋 저장 완료: 344842행, 47개 특성
   📊 타겟 분포 - SELL(0): 5972, HOLD(1): 331133, BUY(2): 7737
   🔍 검증 샘플:
      - 현재가: 9.52
      - 미래 수익률: -0.13%
      - 타겟 라벨: 1 (HOLD)
✅ 전략 시그널 데이터 저장 완료!
📁 저장 위치: /workspace/AI모델/projects/coin/data/1h/final/with_strategies/
✅ 저장 완료: 20개

📊 생성된 퀀트 전략 시그널 요약 (ADA_1h 기준)
이동평균 전략: 4개
  - MA_Cross_3_25_Signal: BUY 3.2%, HOLD 93.5%, SELL 3.2%
  - MA_Trend_Signal: BUY 33.5%, HOLD 34.4%, SELL 32.1%
  - EMA_Cross_5_20_Signal: BUY 3.0%, HOLD 94.0%, SELL 3.0%
  - EMA_Cross_6_24_Signal: BUY 2.5%, HOLD 95.0%, SELL 2.5%

MACD 전략: 6개
  - MACD_Signal: BUY 0.0%, HOLD 0.0%, SELL 0.0%
  - MACD_10_20_Signal: BUY 2.1%, HOLD 95.7%, SELL 2.1%
  - MACD_15_26_Signal: BUY 1.5%, HOLD 97.1%, SELL 1.5%
  - MACD_5_27_Signal: BUY 2.6%, HOLD 94.8%, SELL 2.6%
  - MACD_Zero_Cross_Signal: BUY 1.6%, HOLD 96.7%, SELL 1.6%
  - MACD_Signal_Cross: BUY 3.8%, HOLD 92.5%, SELL 3.8%

RSI 전략: 8개
  - RSI_Reversal_Signal: BUY 3.4%, HOLD 93.5%, SELL 3.2%
  - RSI_Extreme_Signal